<a href="https://colab.research.google.com/github/halen48/SocketsDemo/blob/main/Cliente_Servidores_(Single_Cell)_%5BMultiserver%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socket
from multiprocessing import Process
from threading import Thread, Lock
import time
import random
import pickle
import json

In [ ]:
velocidade = 1.5 #@param {type:"slider", min:0.1, max:2, step:0.1}

#Server

###Autenticação

In [ ]:
class ServerAutenticacao():
  def __init__(self, end, cod, max_clients, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.max_clients = max_clients
    self.lista_clientes = {}
    self.tamanho_buffer = tamanho_buffer
    self.ultimo_id = 0
    self.clientes_conectados = []
    self.blocos = {}
    
  def adicionar_cliente(self, conn, addr):

    id = self.ultimo_id
    self.ultimo_id += 1
    thread = Thread(target = self.parse_cliente, args=(conn, addr, id))
    self.lista_clientes[id] = ( (conn, thread) )
    self.lista_clientes[id][1].start()

    print("[ServerAutenticacao] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))

    return id
  
  def checar_conexoes_clientes(self):
    while True:
      #limpa os clientes que estão desconectados
      self.clientes_conectados = list(self.lista_clientes.keys())
      for c_id in self.clientes_conectados:
        if (not self.lista_clientes[c_id][1].is_alive()):
          self.remover_cliente(c_id)
      #Caso a lista tenha espaço para mais clientes, a rotina encerra por aqui
      if (len(self.lista_clientes) < self.max_clients):
          return
      #Se tem mais clientes que o servidor pode suportar, aguarda...
      print("[ServerAutenticacao] Muitos clientes conectados. Aguardando...")
      print("[ServerAutenticacao - Debug] IDs Conectados: ", self.clientes_conectados)
      time.sleep(0.5/velocidade)

  def gerenciador_clientes(self):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      while True:
        try:
          s.bind(self.endereco)
          s.listen(self.max_clients)
          time.sleep(0.5/velocidade)
          break
        except Exception as e:
          print(e)
          print("Tentando vincular conexão...")
          continue
      
      while True:
        
        self.checar_conexoes_clientes()  

        conn, addr = s.accept()
        print('[ServerAutenticacao] Foi conectado um cliente em:', addr)
        self.adicionar_cliente(conn, addr)
      #s.close()
  
  def remover_cliente(self,index):
    print('[ServerAutenticacao] Desconectando o cliente[%d]: %s'%(index,self.lista_clientes[index][0]))
    self.lista_clientes[index][0].close()
    del self.lista_clientes[index]
    print("[ServerAutenticacao] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))
    
  def connect(self, s, endereco):
    while True:
      try:
        s.connect(endereco)
        return
      except ConnectionRefusedError:
        print('[ServerAutenticacao] Tentando conectar...')
        time.sleep(2)
      except TimeoutError:
        print('\033[3%dm'%(endereco[1]%5)+'[ServerAutenticacao] Timeout! <IP:%d>'%endereco,'\x1b[0m')
        
  
    self.usuario = input("Nome do usuário: ")
  
  def novo_ip(self):
    return ('127.0.0.1', 33334+len(self.blocos))  #trocar

  def parse_cliente(self, conn, addr, id):
    while True:
      try:
        data = conn.recv(self.tamanho_buffer)
        print('\033[3%dm'%(1+id%5)+'[ServerAutenticacao] recebi', repr(data), 'do cliente <id: %d>'%id,'\x1b[0m')
        if not data:
            break
        
        time.sleep(0.5/velocidade)
        conn.sendall(pickle.dumps( (id) )) #envia o ID para o cliente conectado e faz ele esperar a informação chegar


        time.sleep(0.5/velocidade)
        #caso não exista o proximo servidor, cria
        endereco = self.novo_ip()
        s = ServerInterno(endereco, self.codificacao, self.tamanho_buffer, self.endereco, len(self.blocos))
        p = Process(target=s.run)
        p.start()
        
        #envia bloco para o proximo servidor
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as bloco_conn:
          while True:
            try:
              data = pickle.loads(data)
              data = pickle.dumps( (data) )
              self.connect(bloco_conn, endereco)
              bloco_conn.sendall( data )
              #recebe o hash do bloco
              self.blocos["%s:%d"%endereco] = pickle.loads(bloco_conn.recv(self.tamanho_buffer))
              
              break
            except Exception as e:
              print("[ServerAutenticacao]", e)
              time.sleep(0.5/velocidade)
        
        print("[ServerAutenticacao] Servidores conectados:")
        print(json.dumps(self.blocos,indent=4))
        #avisa pro cliente que está ok
        conn.sendall( pickle.dumps("ok") )

      except Exception as e:
        print("[ServerAutenticacao]", e)
        time.sleep(0.5/velocidade)
        continue
    
  
  def run(self):
    self.gerenciador_clientes()


###Servidores Internos

In [ ]:
class ServerInterno():
  def __init__(self, end, cod, tamanho_buffer, endereco_base, id):
    self.endereco = end
    self.codificacao = cod
    self.tamanho_buffer = tamanho_buffer
    self.endereco_mestre = endereco_base
    self.dado = None

  def parse_chain(self,conn, addr):
    while True:
      try:
        data = conn.recv(self.tamanho_buffer)
        print('\033[3%dm'%(1+self.endereco[1]%5)+'[ServerInterno] recebi', repr(data), '\x1b[0m')
        if not data:
            break

        data = pickle.loads( data )

        self.dado = data

        time.sleep(0.5/velocidade)
        conn.sendall(pickle.dumps( "online" )) #avisa que está online

      except Exception as e:
        print("[ServerInterno]", e)
        time.sleep(0.5/velocidade)
        continue
    
  
  def run(self):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      while True:
        try:
          s.bind(self.endereco)
          s.listen(1) #Só recebe informação do mestre
          time.sleep(0.5/velocidade)
          break
        except Exception as e:
          print(e)
          print("Tentando vincular conexão...")
          continue
      
      conn, addr = s.accept()
      print('[ServerInterno] Novo bloco criado. conexão:', addr)
      self.parse_chain(conn, addr)

      


#Client

In [ ]:
class Client():
  def __init__(self, end, cod, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.tamanho_buffer = tamanho_buffer
    self.id = None
    self.usuario = None

  def connect(self, s):
    while True:
      try:
        s.connect(self.endereco)
        break
      except ConnectionRefusedError:
        #print('[Cliente] Tentando conectar...')
        time.sleep(2)
      except TimeoutError:
        if(self.id):
          print('\033[3%dm'%(1+self.id%5)+'[Cliente] Timeout! <id:%d>'%self.id,'\x1b[0m')
        else:
          print("[Cliente] Timeout! <id: Não conectado>")
  
    self.usuario = input("Nome do usuário: ")


  def rotina(self):
  
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      self.connect(s)
      while True:
        time.sleep(1/velocidade)
        
        msg = input("Mensagem: ")
        data = {'Mensagem':msg,'Usuario':self.usuario}
        #Envia a mensagem criada
        print("[Cliente] Enviando", json.dumps(data,indent=4))
        s.sendall( pickle.dumps( data ) )
        
        #espera receber um ID do servidor
        data = pickle.loads ( s.recv(self.tamanho_buffer) )
        if( not self.id):
          self.id = data
        print('\033[3%dm'%(1+self.id%5)+'[Cliente] Mensagem enviada <id:%d>'%self.id,'\x1b[0m')
        
        print('\033[3%dm'%(1+self.id%5)+'[Cliente] Aguardando resposta do servidor... <id:%d>'%self.id,'\x1b[0m')
        data = pickle.loads ( s.recv(self.tamanho_buffer) )
        if(data == "ok"):
          print('\033[3%dm'%(1+self.id%5)+'[Cliente] Informação processada pelo servidor! <id:%d>'%self.id,'\x1b[0m')


  
  def run(self):
    self.rotina()

#Rotina

In [ ]:
endereco = ('127.0.0.1', 33333)
codificacao = 'latin-1'
tamanho_buffer = 2048

s = ServerAutenticacao(endereco,codificacao, 5, tamanho_buffer)

p = Process(target=s.run)
p.start()

Thread(target = Client(endereco,codificacao, tamanho_buffer).run ).start()
  
p.join()